In [53]:
import pandas as pd
import pickle
import numpy as np
import networkx as nx
import re
import time

In [73]:
df_ivs = pickle.load(open('data/ivs_exploded_100.p', 'rb'))
# df_ivs_old = pickle.load(open('data/ivs_exploded_100_old.p', 'rb'))
# routes_dests = pickle.load(open('data/users_ship_specific_routes.p', 'rb'))
# df_ships = pd.read_excel('data/ship_types.xlsx')

# G = pickle.load(open("data/network.p", "rb"))
# paths = pickle.load(open("data/paths.p", "rb"))
# df_random = pickle.load(open("data/df_random.p", "rb"))
# df_abm = pickle.load(open("data/df_abm.p", "rb"))
# non_zero_flows = pickle.load(open("data/non_zero_flows.p", "rb"))
# feasible_combinations = pickle.load(open('data/feasible_comb.p', 'rb'))
# optimal_flows = pickle.load(open('data/optimal_flows.p', 'rb'))
# path_lengths = pickle.load(open('data/path_lengths_ship_specific_routes.p', 'rb'))
agent_data = pickle.load(open('data/agent_data.p', 'rb'))
model_data = pickle.load(open('data/model_data.p', 'rb'))

In [78]:
df_random.trip_count.sum()

26756

In [81]:
(26756/365)*7

513.1287671232876

## Process agent data

### Vessel data

In [55]:
agent_data = agent_data.reset_index()

In [56]:
agent_data = agent_data.loc[agent_data.Step>(24*60)]

In [57]:
vessel_data = agent_data.loc[agent_data.vessel_status.notnull()]
vessel_data = vessel_data.drop(columns=['removed_at'])
vessel_data['status'] = 0
vessel_data['at_station'] = 0
vessel_data.status = vessel_data.vessel_status.apply(lambda x: x[0] if type(x) == list else x)
vessel_data.at_station = vessel_data.vessel_status.apply(lambda x: x[1] if type(x) == list else x)

In [58]:
vessel_data_dict = {}
def process_vessel_data(x):
    if not x.AgentID in vessel_data_dict.keys():
        vessel_data_dict[x.AgentID] = {"charging":{}, "inline":{}, "driving":0}
    if x.status == 'driving':
        vessel_data_dict[x.AgentID]['driving'] += 1
    else:
        if not x.at_station in vessel_data_dict[x.AgentID][x.status].keys():
            vessel_data_dict[x.AgentID][x.status][x.at_station] = 1
        else:
            vessel_data_dict[x.AgentID][x.status][x.at_station] += 1

In [59]:
vessel_data.apply(lambda x: process_vessel_data(x), axis=1)

326148     None
326149     None
326150     None
326151     None
326152     None
           ... 
2631387    None
2631388    None
2631389    None
2631390    None
2631391    None
Length: 168496, dtype: object

In [60]:
df = pd.DataFrame(vessel_data_dict)
df = df.transpose()

In [61]:
df['total_charging'] = 0
df.total_charging = df.charging.apply(lambda x: sum(x.values()))
df['total_inline'] = 0
df.total_inline = df.inline.apply(lambda x: sum(x.values()))

In [62]:
df1 = vessel_data.loc[vessel_data.status != 0].loc[:,['AgentID','Step','vessel_route','departed_from', 'combi','battery_size', 'generated_at']].groupby('AgentID').first()
vessel_data = pd.merge(df1, df, left_index=True, right_index=True)

In [63]:
vessel_data = vessel_data.loc[vessel_data.generated_at > (24*60)]
vessel_data
# looks okay! Final check with pycharm later

,Step,vessel_route,departed_from,combi,battery_size,generated_at,charging,inline,driving,total_charging,total_inline
AgentID,,,,,,,,,,,
73,1444,"(NLHTB, NLBZM, 0)",8866775,"[8866893, 8862801, 8866775]",6650.0,1443.0,"{'8866893': 115, '8862801': 155}",{},549,270,0
74,1475,"(NLGOR, NLRTM, 0)",8863306,"[8863019, 8863306]",3220.0,1474.0,{'8863019': 66},{},237,66,0
75,1482,"(NLAER, NLNIE, 0)",8867538,"[8864005, 8863360, 107]",6650.0,1481.0,"{'8863360': 135, '107': 60, '8864005': 155}",{},471,350,0
76,1484,"(NLTNZ, NLVLI, 0)",8864111,[8863469],6650.0,1483.0,{'8863469': 143},{},110,143,0
77,1484,"(NLGOR, NLRTM, 0)",8863019,"[8863019, 8863306]",9404.0,1483.0,{'8863306': 192},{},237,192,0
...,...,...,...,...,...,...,...,...,...,...,...
530,11431,"(NLRTM, NLAPN, 0)",8863360,"[8863306, 8863360]",2031.0,11430.0,{},{},90,0,0
531,11456,"(NLOOS, NLTLB, 0)",8863818,[8863818],6650.0,11455.0,{},{},65,0,0
532,11464,"(NLAMS, NLZAA, 0)",8866889,[8866889],3220.0,11463.0,{},{},30,0,0


In [72]:
vessel_data.groupby("vessel_route").mean()

,Step,battery_size,generated_at,total_charging,total_inline
vessel_route,,,,,
"(NLABL, NLRTM, 0)",5921.375000,6565.500000,5920.375000,29.500000,0.000000
"(NLAER, NLLEY, 0)",6955.500000,2149.900000,6954.500000,29.600000,0.000000
"(NLAER, NLNIE, 0)",5021.000000,5159.100000,5020.000000,196.900000,1.700000
"(NLAER, NLRTM, 0)",6544.466667,6514.800000,6543.466667,298.600000,4.200000
"(NLALK, NLIJM, 0)",5921.000000,3106.875000,5920.000000,71.875000,0.000000
"(NLAMS, NLAER, 0)",6627.560976,2551.682927,6626.560976,49.146341,2.121951
"(NLAMS, NLMOE, 3)",5067.500000,6650.000000,5066.500000,372.916667,0.666667
"(NLAMS, NLUTC, 1)",5921.600000,6239.400000,5920.600000,150.200000,0.000000
"(NLAMS, NLWMO, 0)",4089.000000,1400.000000,4088.000000,5.250000,0.000000


### Charging station data

In [68]:
df1 = pd.DataFrame(agent_data.loc[(agent_data.charging_stations>0)].groupby('AgentID').first().loc[:,'charging_stations'])

In [69]:
df2 = pd.DataFrame(agent_data.loc[(agent_data.charging_stations>0)].groupby('AgentID').station_status.mean())

In [70]:
cs_data = pd.merge(df1, df2, left_index=True, right_index=True)

In [71]:
cs_data
# TODO check why some are not visited at all within 7 days, seems off

,charging_stations,station_status
AgentID,,
104,2.0,0.172173
107,2.0,0.159573
113,2.0,0.069097
122,1.0,0.073611
22638146,1.0,0.057837
22638327,1.0,0.113889
30984595,2.0,0.118353
8862663,2.0,0.170536
8862801,1.0,0.266369


### Remark
Something seems off, a lot of stations are not used at all
Way forward:
1. Check whether this may be clarified based on input data
2. Recheck logic behind generation
3. Check whether the generation is going well and as expected